In [2]:
### minimize training data ###
import pandas as pd
import random as rndn

filename = r"../input/train.csv"
n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
s = 100000 #desired sample size
skip = sorted(rndn.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
df = pd.read_csv(filename, skiprows=skip)
df.to_csv("../input/train_min.csv")


In [2]:
import pandas as pd

df_train = pd.read_csv("../input/train_min.csv")
df_train.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,0,2010-03-27 02:26:39.0000001,14.5,2010-03-27 02:26:39 UTC,-73.961598,40.716487,-73.960417,40.674963,2
1,1,2015-02-15 22:41:32.0000003,9.0,2015-02-15 22:41:32 UTC,-74.003448,40.733101,-74.008003,40.707214,1
2,2,2011-01-14 18:10:00.000000129,5.7,2011-01-14 18:10:00 UTC,-73.986370,40.747380,-73.975567,40.744572,2
3,3,2011-06-20 14:52:22.0000004,18.5,2011-06-20 14:52:22 UTC,-73.997892,40.724297,-73.977578,40.786941,1
4,4,2011-10-14 12:20:00.00000051,4.9,2011-10-14 12:20:00 UTC,-73.952797,40.780497,-73.955042,40.788527,5


In [ ]:
df_test = pd.read_csv("../input/test.csv")
df_test.head()

In [ ]:
### data preprocessing ###
### copy for submission ###
df_test_for_sub = df_test["key"]
df_test_for_sub

In [ ]:
# delete the fares that are less than 0
df_train = df_train.loc[df_train.fare_amount > 0]
df_train.fare_amount.min()

In [ ]:
len(df_train)

In [ ]:
train_X = df_train.drop(columns=['fare_amount'])

Y = df_train['fare_amount']
#Y = np.log(df_train.fare_amount)

test_X = df_test

In [ ]:
df = train_X.append(test_X, sort=False)
df.head()

In [ ]:
### calculate the distance traveled ###

import numpy as np

def haversine_distance(df, lat1, long1, lat2, long2):
    """
    Calculates the haversine distance between 2 sets of GPS coordinates in df
    """
    r = 6371  # average radius of Earth in kilometers
       
    phi1 = np.radians(df[lat1])
    phi2 = np.radians(df[lat2])
    
    delta_phi = np.radians(df[lat2]-df[lat1])
    delta_lambda = np.radians(df[long2]-df[long1])
     
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = (r * c) # in kilometers

    return d

In [ ]:
df['dist_km'] = haversine_distance(
    df,
    'pickup_latitude', 
    'pickup_longitude', 
    'dropoff_latitude', 
    'dropoff_longitude'
)

df.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


In [4]:
### data preprocessing ###
### copy for submission ###
df_test_for_sub = df_test["key"]
df_test_for_sub

0       2015-01-27 13:08:24.0000002
1       2015-01-27 13:08:24.0000003
2       2011-10-08 11:53:44.0000002
3       2012-12-01 21:12:12.0000002
4       2012-12-01 21:12:12.0000003
                   ...             
9909    2015-05-10 12:37:51.0000002
9910    2015-01-12 17:05:51.0000001
9911    2015-04-19 20:44:15.0000001
9912    2015-01-31 01:05:19.0000005
9913    2015-01-18 14:06:23.0000006
Name: key, Length: 9914, dtype: object

In [5]:
# delete the fares that are less than 0
df_train = df_train.loc[df_train.fare_amount > 0]
df_train.fare_amount.min()

1.5

In [6]:
len(df_train)

99990

In [7]:
train_X = df_train.drop(columns=['fare_amount'])

Y = df_train['fare_amount']
#Y = np.log(df_train.fare_amount)

test_X = df_test

In [8]:
df = train_X.append(test_X, sort=False)
df.head()

,Unnamed: 0,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,0.0,2010-03-27 02:26:39.0000001,2010-03-27 02:26:39 UTC,-73.961598,40.716487,-73.960417,40.674963,2
1,1.0,2015-02-15 22:41:32.0000003,2015-02-15 22:41:32 UTC,-74.003448,40.733101,-74.008003,40.707214,1
2,2.0,2011-01-14 18:10:00.000000129,2011-01-14 18:10:00 UTC,-73.986370,40.747380,-73.975567,40.744572,2
3,3.0,2011-06-20 14:52:22.0000004,2011-06-20 14:52:22 UTC,-73.997892,40.724297,-73.977578,40.786941,1
4,4.0,2011-10-14 12:20:00.00000051,2011-10-14 12:20:00 UTC,-73.952797,40.780497,-73.955042,40.788527,5


In [11]:
### calculate the distance traveled ###

import numpy as np

def haversine_distance(df, lat1, long1, lat2, long2):
    """
    Calculates the haversine distance between 2 sets of GPS coordinates in df
    """
    r = 6371  # average radius of Earth in kilometers
       
    phi1 = np.radians(df[lat1])
    phi2 = np.radians(df[lat2])
    
    delta_phi = np.radians(df[lat2]-df[lat1])
    delta_lambda = np.radians(df[long2]-df[long1])
     
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = (r * c) # in kilometers

    return d

In [12]:
df['dist_km'] = haversine_distance(
    df,
    'pickup_latitude', 
    'pickup_longitude', 
    'dropoff_latitude', 
    'dropoff_longitude'
)

df.head()

,Unnamed: 0,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dist_km
0,0.0,2010-03-27 02:26:39.0000001,2010-03-27 02:26:39 UTC,-73.961598,40.716487,-73.960417,40.674963,2,4.618332
1,1.0,2015-02-15 22:41:32.0000003,2015-02-15 22:41:32 UTC,-74.003448,40.733101,-74.008003,40.707214,1,2.903933
2,2.0,2011-01-14 18:10:00.000000129,2011-01-14 18:10:00 UTC,-73.986370,40.747380,-73.975567,40.744572,2,0.962144
3,3.0,2011-06-20 14:52:22.0000004,2011-06-20 14:52:22 UTC,-73.997892,40.724297,-73.977578,40.786941,1,7.172769
4,4.0,2011-10-14 12:20:00.00000051,2011-10-14 12:20:00 UTC,-73.952797,40.780497,-73.955042,40.788527,5,0.912682
